# Dataset Generator

In [2]:
%load_ext autoreload
%autoreload 2
import os
import random
from scipy.spatial.transform import Rotation as R
import mujoco
import mujoco.renderer
import numpy as np
from sensors import Sensors
from vectorized_point_cloud import VectorizedPC

Loading the model names listed in a given directory.

In [3]:
models_paths = os.listdir('ycb')

# models = [models_paths[i] for i in range(3)]

models = models_paths

## Functionality Methods

In [4]:
def new_xml(model, quat):
    xml =  f"""
    <mujoco>
        <asset>
            <mesh name="{model}" file="ycb/{model}" scale="1 1 1"/>
        </asset>
        <worldbody>
            <light name="top" pos="0 0 1"/>
            <geom pos="0 0 0" type="mesh" contype="0" conaffinity="0" group="1" density="0" mesh="{model}"
            quat="{quat[0]} {quat[1]} {quat[2]} {quat[3]}"
            />
                        
            <camera name="camera1" pos="-1 0.1 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera2" pos="1 0.1 0" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera3" pos="1 0.3 0.5" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera4" pos="1 0.1 -0.2" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
              
        </worldbody>
    </mujoco>
    """

    return xml

In [5]:
def get_elements_with_probability(arr=[0,1,2,3]):
    # Define the probabilities
    probabilities = [0.7, 0.25, 0.05]

    # Determine the number of elements to select based on probabilities
    num_elements = random.choices([1, 2, 3], probabilities)[0]

    # Shuffle the array to randomize the selection
    random.shuffle(arr)

    # Return a set of the selected elements
    return arr[:num_elements]

## Main Generation Loop

In [7]:
# Dataset Generator Configuration
dataset_size = 100_000

sensors = Sensors()

pc = VectorizedPC((480, 640), 60)

quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)
m = mujoco.MjModel.from_xml_string(new_xml(models[0], quat))

cameras = [0,1,3]

rot_matrices = []
positions = []
for i in cameras:
    rot_matrices.append(R.from_quat(m.cam(i).quat).as_matrix())
    positions.append(m.cam_pos[i])

for model_name in models:
    name = model_name.split('.')[0]
    for n in range(dataset_size // len(models)):
        quat = np.random.random(4)
        quat = quat / np.linalg.norm(quat)

        # Generating new xml
        xml = new_xml(model_name, quat)

        # Create the model and data objects.
        m = mujoco.MjModel.from_xml_string(xml)
        d = mujoco.MjData(m)

        # Make renderer, render and show the pixels
        depth = mujoco.Renderer(m,480, 640); depth.enable_depth_rendering()
        # sgmnt = mujoco.Renderer(m,480, 640); sgmnt.enable_segmentation_rendering()

        # Doing initial loading
        mujoco.mj_step(m, d)

        # cameras = get_elements_with_probability([0,1,2,3])

        depth = [*sensors.get_depth_image_matrices(m, d, depth, cameras).values()]

        # rot_matrices = []
        # positions = []
        # for i in cameras:
        #     rot_matrices.append(R.from_quat(m.cam(i).quat).as_matrix())
        #     positions.append(m.cam_pos[i])

        mat = np.empty((0,3))
        for i in range(len(cameras)):
            mat = np.concatenate((mat, 
                                  pc.get_points(depth[i], 
                                                rot_matrices[i], 
                                                positions[i])))
            
        np.save(f'../dataset/v5/{name}_{"%06d" % (n+1,)}.npy', mat)

## Point Cloud Visualization

In [8]:
point_clouds = os.listdir('../dataset/v5')

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_point_cloud(mat):
    # Create a 3D figure
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # Define the x, y, z coordinates of the point cloud
    x = mat[:, 0]
    y = mat[:, 1]
    z = mat[:, 2]

    # Plot the point cloud data
    ax.scatter(x, y, z, s=1)

    # Set the axis labels
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    ax.scatter(0, 0, 0, s=10)

    # ax.set_xlim(-0.3, 0.3)
    # ax.set_ylim(-0.3, 0.3)
    # ax.set_zlim(-0.3, 0.3)

    # Show the plot
    plt.show()
    # print

import open3d as o3d

def plot_point_cloud_open3d(mat):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(mat)
    o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
for i in range(10):
    mat = np.load(f'../dataset/v4/{point_clouds[i]}')
    print(point_clouds[i])
    print(mat.shape)
    plot_point_cloud_open3d(mat)

003_cracker_box_000001.npy
(16159, 3)
037_scissors_000001.npy
(2683, 3)
070-b_colored_wood_blocks_000001.npy
(482, 3)
073-b_lego_duplo_000001.npy
(725, 3)
013_apple_000001.npy
(2644, 3)
057_racquetball_000001.npy
(1359, 3)
073-e_lego_duplo_000001.npy
(1546, 3)
072-e_toy_airplane_000001.npy
(492, 3)
073-f_lego_duplo_000001.npy
(745, 3)
032_knife_000001.npy
(1202, 3)


In [ ]:
import numpy as np

choices = np.array([0,1,2,3,4,5,6,7,8,9])
chosen_index = np.random.choice(choices, 3, replace=False)

choices[[1,2,3]]

In [4]:
import open3d as o3d
import numpy as np

# Generate 1000 random points in a sphere
points = np.random.rand(100, 3)
# Repeat same color for 100 points
colors = np.tile([255, 0, 0], (100, 1))

# Generate another 1000 random points in a sphere
points_2 = np.random.rand(100, 3)
# Repeat same color for 100 points
colors_2 = np.tile([0, 255, 0], (100, 1))

# Concatenate the points and colors
points = np.vstack((points, points_2))
colors = np.vstack((colors, colors_2))

# Create a point cloud from the points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])
